## multipal models with pipeline and columnTransformer



In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [6]:
df=sns.load_dataset("tips")

In [7]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [8]:
## total bill is target variable# encoding ,missing value treatment ,scaling ,>>automate
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [9]:
x=df.drop("total_bill",axis=1)
y=df["total_bill"]

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1)

In [14]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [15]:
#Handling the missing value
# data encoding
# feature scaling


In [21]:
from sklearn.impute import SimpleImputer #for missing vlaue
from sklearn.preprocessing import OneHotEncoder #for encoding
from sklearn.preprocessing import StandardScaler #for scaling

from sklearn.pipeline import Pipeline # a sequence of data transformer
from sklearn.compose import ColumnTransformer # Groups all the pipeline steps for each of the column

In [22]:
cat_cols = ["sex", "smoker", "day", "time"]
num_cols = ["tip", "size"]

In [23]:
#feature engineering automation using pipeline and columntransformer

num_pipeline = Pipeline(steps = [('imputation', SimpleImputer(strategy = "median")),
                                ('scaling', StandardScaler())])
cat_pipeline = Pipeline(steps = [('imputation', SimpleImputer(strategy = "most_frequent")),
                                ('encoding', OneHotEncoder())])

In [28]:
preprocessor=ColumnTransformer([("num_pipeline",num_pipeline,num_cols),
                                 ("cat_pipeline",cat_pipeline,cat_cols)])

In [29]:
x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)

In [30]:
x_train

array([[-1.47443803, -0.57766863,  1.        , ...,  0.        ,
         1.        ,  0.        ],
       [-0.71612531,  1.47042924,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.19880579,  1.47042924,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 0.43283335, -0.57766863,  1.        , ...,  1.        ,
         0.        ,  1.        ],
       [-1.29060464, -0.57766863,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.1034652 ,  0.44638031,  1.        , ...,  0.        ,
         1.        ,  0.        ]])

In [31]:
x_test

array([[-1.48209775, -1.60171757,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ],
       [ 0.04984713, -0.57766863,  1.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         1.        ,  0.        ],
       [ 0.36389583,  0.44638031,  0.        ,  1.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ],
       [-0.33313909, -0.57766863,  0.        ,  1.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  1.        ],
       [ 0.04984713, -0.57766863,  0.        ,  1.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  1.        ],
       [-1.14506988, -0.57766863,  0.        ,  1.        ,  0.        ,
         1.        ,  

In [32]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [33]:
models = {"support vector Regressor": SVR(),
         "DT Regressor": DecisionTreeRegressor(),
          "Multiple Linear regression": LinearRegression(),
         "Random_forest_Regressor": RandomForestRegressor()}

In [38]:
from sklearn.metrics import r2_score
def model_train_eval(x_train, y_train, x_test, y_test, models):
    evaluation = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        model_score = r2_score(y_test, y_pred)
        evaluation[list(models.keys())[i]] = model_score
    return evaluation

In [39]:
model_train_eval(x_train, y_train, x_test, y_test, models)

{'support vector Regressor': 0.4069433055293954,
 'DT Regressor': 0.17110196270037903,
 'Multiple Linear regression': 0.6719006761625259,
 'Random_forest_Regressor': 0.6442098250864795}

In [40]:
from sklearn.ensemble import RandomForestRegressor

In [41]:
rfr=RandomForestRegressor(oob_score=True)

In [42]:
rfr

RandomForestRegressor(oob_score=True)

In [43]:
from sklearn.model_selection import RandomizedSearchCV
params={'max_depth':[1,50, 100, 150,200],
        'n_estimators':[50, 100,200]}

In [44]:
params

{'max_depth': [1, 50, 100, 150, 200], 'n_estimators': [50, 100, 200]}

In [48]:
reg=RandomizedSearchCV(rfr,param_distributions=params,cv=5,verbose=3,scoring='r2')

In [49]:
reg

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   param_distributions={'max_depth': [1, 50, 100, 150, 200],
                                        'n_estimators': [50, 100, 200]},
                   scoring='r2', verbose=3)

In [50]:
reg.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END ...max_depth=100, n_estimators=100;, score=0.318 total time=   0.2s
[CV 2/5] END ...max_depth=100, n_estimators=100;, score=0.489 total time=   0.2s
[CV 3/5] END ...max_depth=100, n_estimators=100;, score=0.228 total time=   0.2s
[CV 4/5] END ...max_depth=100, n_estimators=100;, score=0.573 total time=   0.2s
[CV 5/5] END ...max_depth=100, n_estimators=100;, score=0.387 total time=   0.2s
[CV 1/5] END ....max_depth=150, n_estimators=50;, score=0.282 total time=   0.0s
[CV 2/5] END ....max_depth=150, n_estimators=50;, score=0.496 total time=   0.0s
[CV 3/5] END ....max_depth=150, n_estimators=50;, score=0.201 total time=   0.0s
[CV 4/5] END ....max_depth=150, n_estimators=50;, score=0.616 total time=   0.0s
[CV 5/5] END ....max_depth=150, n_estimators=50;, score=0.346 total time=   0.0s
[CV 1/5] END ....max_depth=50, n_estimators=200;, score=0.307 total time=   0.5s
[CV 2/5] END ....max_depth=50, n_estimators=200;

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   param_distributions={'max_depth': [1, 50, 100, 150, 200],
                                        'n_estimators': [50, 100, 200]},
                   scoring='r2', verbose=3)

In [51]:
reg.best_params_

{'n_estimators': 100, 'max_depth': 100}

In [52]:
reg.best_score_

0.3990769473749572